In [2]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 


import bk.load
import bk.signal
import bk.plot


import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [111]:
bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
bk.load.session = 'Rat08-20130713'


session = 'Rat08-20130713'
neurons,shanks = bk.load.loadSpikeData(bk.load.path)
pos = bk.load.pos()
states = bk.load.states()
run = scipy.io.loadmat(bk.load.path + 'runintervals.mat')
run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')



Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-NeuronsShanks.npy


In [117]:
run = scipy.io.loadmat(bk.load.path + '//' +bk.load.session+'-TrackRunTimes.mat')
run = nts.IntervalSet(run['trackruntimes'][:,0],run['trackruntimes'][:,1],time_units='s')

In [102]:
pos_run = pos.restrict(run)

In [103]:
pos_spike.as_units('s')

,x,y
Time (s),,
2.004900,118.1081,44.97370
2.035699,118.1081,44.97370
2.049550,118.1081,44.97370
4.535450,118.1081,44.97370
11.840000,198.7073,30.96000
...,...,...
22506.817600,231.3400,31.82000
22506.823000,231.3400,31.82000
22506.829450,231.3400,31.82000


In [104]:
xpos = pos_run.x
ypos = pos_run.y

In [105]:
pos_run.x.as_units('s')

Time (s)
9.1136        118.7875
9.1392        119.4884
9.1648        120.1678
9.1904        120.8730
9.2160        121.5782
                ...   
22670.5152    234.7800
22670.5408    234.7800
22670.5664    234.7800
22670.5920    234.7800
22670.6176    234.7800
Name: x, Length: 29432, dtype: float64

In [108]:
dt = 1/100
for i in range(50):
    pos_spike = pos_run.realign(neurons[i])
    xbins = np.linspace(xpos.min(), xpos.max()+0.01, 25)
    ybins = np.linspace(ypos.min(), ypos.max()+0.01, 25)
    spike_count2,_,_ = np.histogram2d(pos_spike['y'], pos_spike['x'], [ybins,xbins])
    occupancy2, _, _ = np.histogram2d(pos['y'], pos['x'], [ybins,xbins])
    spike_count2 = spike_count2/(occupancy2+1)
    place_field = spike_count2/dt
    place_field = pd.DataFrame(index = ybins[0:-1]+np.diff(ybins)/2,columns = xbins[0:-1]+np.diff(xbins)/2,data = place_field)


    # And plot everything
    plt.figure()
    plt.subplot(211)

    plt.plot(pos['x'], pos['y'], alpha = 0.5)
    plt.scatter(pos_spike['x'], pos_spike['y'], color = 'red', alpha = 0.5, edgecolor = None)
    for i in xbins: plt.axvline(i)
    for i in ybins: plt.axhline(i)
    plt.xlim(xbins[0], xbins[-1])
    plt.ylim(ybins[0], ybins[-1])
    plt.subplot(212)
    plt.imshow(place_field, origin = 'lower', aspect= 'auto')

/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [109]:
ypos.max()

110.94

In [126]:
# for i in run.as_units('s').iloc():
#     print(i.end)
    
    
ypos.as_units('s')[206:207]

Time (s)
206.0032     41.7100
206.0288     41.7100
206.0544     41.7100
206.0800     41.7100
206.1056     41.7100
206.1312     41.7100
206.1568     41.7100
206.1824     41.7100
206.2080     41.7100
206.2336     41.7100
206.3616    108.3600
206.3872    108.3600
206.4128    108.3600
206.4384    108.3600
206.4640    108.3600
206.4896    108.3600
206.5152    108.3600
206.5408    108.3600
206.5664    108.3600
206.5920    108.3600
206.6176    108.3600
206.6432    108.3600
206.6688    108.3600
206.6944    108.3600
206.7200    108.3600
206.7456    108.3600
206.7712    108.3600
206.7968    108.3600
206.8224    108.3600
206.8480    108.3600
206.8736    108.3600
206.8992    108.3600
206.9248    108.3600
206.9504    108.2740
206.9760    108.1364
Name: y, dtype: float64

In [113]:
run.as_units('s')

,start,end
0,0.0,206.3
1,9405.5,12649.0
2,22415.0,22680.0
